# Módulo 3 - Seleção de Modelos de Aprendizado de Máquina

# Desafio do Módulo 3

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ilpd = pd.read_csv('https://www.openml.org/data/get_csv/1590565/phpOJxGL9')

In [ ]:
ilpd.shape

(583, 11)

In [ ]:
ilpd.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,Class
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [ ]:
#Convertendo o atributo categórigo da V2 em numérico:
#0 = Female
#1 = Male

name_to_class = {'Female': 0, 'Male': 1}

#substituindo os valores categóricos pelo mapeamento
ilpd['V2'] = ilpd['V2'].map(name_to_class)

#Conferindo a alteração
ilpd.head(5)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,Class
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [ ]:
#Verificando valores faltantes

ilpd.isnull().sum()

#Não existem valores faltantes

V1       0
V2       0
V3       0
V4       0
V5       0
V6       0
V7       0
V8       0
V9       0
V10      0
Class    0
dtype: int64

In [ ]:
ilpd.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,Class
count,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000
mean,44.746141,0.756432,3.298799,1.486106,290.576329,80.713551,109.910806,6.483190,3.141852,0.947064,1.286449
std,16.189833,0.429603,6.209522,2.808498,242.937989,182.620356,288.918529,1.085451,0.795519,0.318492,0.452490
min,4.000000,0.000000,0.400000,0.100000,63.000000,10.000000,10.000000,2.700000,0.900000,0.300000,1.000000
25%,33.000000,1.000000,0.800000,0.200000,175.500000,23.000000,25.000000,5.800000,2.600000,0.700000,1.000000
50%,45.000000,1.000000,1.000000,0.300000,208.000000,35.000000,42.000000,6.600000,3.100000,0.947064,1.000000
75%,58.000000,1.000000,2.600000,1.300000,298.000000,60.500000,87.000000,7.200000,3.800000,1.100000,2.000000
max,90.000000,1.000000,75.000000,19.700000,2110.000000,2000.000000,4929.000000,9.600000,5.500000,2.800000,2.000000


In [ ]:
#Libs que serão usadas
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.metrics import f1_score, make_scorer

In [ ]:
y = np.array(ilpd['Class'])

x = np.array(ilpd.drop('Class', axis=1))

In [ ]:
#instanciando o classificador SVC
clf_svc = SVC()

#definindo o tipo de validação cruzada e o número de folds
cv_strat_svc = StratifiedKFold(n_splits=10)

#definindo a estratégia de score a partir da métrica f1
f1_svc = make_scorer(f1_score)

#definindo hiperparâmetros
hiperparam = {'kernel':('sigmoid', 'rbf'), 'C':[1, 10]}

#instanciando e modelando o random search com os hiperparametros e a validação definidas.
random_cv_svc = RandomizedSearchCV(clf_svc, hiperparam, cv=cv_strat_svc, scoring=f1_svc, random_state=42, n_iter=5)
random_cv_svc.fit(x, y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 4 is smaller than n_iter=5. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=5, n_jobs=None,
                   param_distributions={'C': [1, 10],
                                        'kernel': ('sigmoid', 'rbf')},
                   pre_dispatch='2*n_jobs', random_state=42, refit=True,
                   return_train_score=False, scoring=make_scorer(f1_score),
                   verbose=0)

In [ ]:
#apresentando os melhores resultados encontrados pelo Random Search com o SVC
print('Melhor resultado f1:', random_cv_svc.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', random_cv_svc.best_params_)
print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', random_cv_svc.best_estimator_)

Melhor resultado f1: 0.832818081808181


Melhor configuração de hiperparâmetros: {'kernel': 'rbf', 'C': 1}


Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: 
 SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [ ]:
#Repetindo usando random forest
clf_rf = RandomForestClassifier(random_state=42)

#definindo o tipo de validação cruzada e o número de folds
cv_strat_rf = StratifiedKFold(n_splits=10)

#definindo a estratégia de score a partir da métrica f1
f1_rf = make_scorer(f1_score)

#Hiperparâmetros
hiperparam1 = {'n_estimators':randint(100, 200), 'bootstrap': (True, False), 'criterion': ('gini', 'entropy')}

random_cv_rf = RandomizedSearchCV(clf_rf, hiperparam1, cv=cv_strat_rf, scoring=f1_rf, random_state=42, n_iter=5)
random_cv_rf.fit(x, y)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_s...
                     

In [ ]:
#apresentando os melhores resultados encontrados pelo Random Search com o Random Forest
print('Melhor resultado f1:', random_cv_rf.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', random_cv_rf.best_params_)
print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', random_cv_rf.best_estimator_)

Melhor resultado f1: 0.8019647527623143


Melhor configuração de hiperparâmetros: {'bootstrap': True, 'criterion': 'gini', 'n_estimators': 171}


Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: 
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=171,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)


# Fim

# Visite o meu [github](https://github.com/k3ybladewielder) <3